In [1]:
import collections
import math
import random

### Get a sieve of all numbers $n$ to indicate whether they are prime, with $0 \leq n \leq 2\times10^7$

In [2]:
def get_primes_eratosthenes_sieve(n):
    is_prime = [1] * (n + 1)
    p = 2
    is_prime[0] = 0
    is_prime[1] = 0
    while p * p <= n:
        if is_prime[p]:
            for i in range(p * 2, n + 1, p):
                is_prime[i] = 0
        p += 1
    prime_list = []
    for i in range(n + 1):
        if is_prime[i]:
            prime_list.append(i)
    return is_prime

is_prime = get_primes_eratosthenes_sieve(2 * 10 ** 7)

### Use Pollard's Rho algorithm for finding prime factorizations for any $n$

In [3]:
# https://github.com/zhangbo2008/python_algorithm2/blob/c53669703b957a079f100c12711f86f5fc2f9389/algorithms/factorization/pollard_rho.py
def pollard_rho_prime_factorization(x):
    def f(x):
        return x * x + 1

    def rho(n, x1=2, x2=2):
        if n % 2 == 0:
            return 2
        i = 0
        while True:
            x1 = f(x1) % n
            x2 = f(f(x2)) % n
            divisor = math.gcd(abs(x1 - x2), n)
            i += 1
            if divisor != 1:
                break
            if i > 500:
                x1 = random.randint(1, 10)
                x2 = random.randint(1, 10)
                i = 0
        return divisor

    def pollard_rho_rec(x, factors):
        if x == 1:
            return

        if is_prime[x]:
            factors.append(x)
            return

        divisor = rho(x, random.randint(1, 10), random.randint(1, 10))
        pollard_rho_rec(divisor, factors)
        pollard_rho_rec(x // divisor, factors)

    if x == 1 or x == 0:
        return {x: 1}
    factors = []
    pollard_rho_rec(x, factors)
    return collections.Counter(factors)

### Store prime factorizations for each $n$, with $1\leq n\leq N$ for rapid access 

In [4]:
N = 2 * 10 ** 7
prime_factorization = {n: pollard_rho_prime_factorization(n) for n in range(1, N + 1)}

### Compute binomial coefficients $20000000 \choose k$ iteratively, for $1\leq k\leq15000000$, and keep track of their prime factorizations in each iteration

In [5]:
K = 15 * 10 ** 6
binomial_coefficient, binomial_pf = 1, dict()
for k in range(1, K + 1):
    # (n, k) = (n, k - 1) * (n - k + 1) / k
    multiply_pf, divide_pf = prime_factorization[N - k + 1], prime_factorization[k]
    for p in multiply_pf:
        if p in (0, 1):
            continue
        if p in binomial_pf:
            binomial_pf[p] += multiply_pf[p]
        else:
            binomial_pf[p] = multiply_pf[p]
            
    for p in divide_pf:
        if p in (0, 1):
            continue
        # Guaranteed to have p in binomial_pf
        binomial_pf[p] -= divide_pf[p]
        
sum_of_terms_in_pf = sum(p * binomial_pf[p] for p in binomial_pf)
sum_of_terms_in_pf

7526965179680